# Differential Privacy for BotnetIot Dataset
*Project Background:* <br>
We, as a student, has a labeled private dataset and an unlabeled public dataset. We would like to collaberate with certain number of teachers to label our public dataset by training their similar datasets. In order to protect the data privacy of the teachers, we agree to add global noise to the labels they return for the public dataset. Then we label our public dataset based on the labels that most teachers agree. We combine the new labels with the public dataset and train with our private dataset. <br>

*PATE analysis* is performed on teachers' predictions and the new labels with certain level of epsilon indicating how much information leakage is allowed.

In [ ]:
!pip install imblearn

In [ ]:
# %%capture
# !pip install syft

In [ ]:
# import necessary packages
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch
from torchvision import datasets, transforms
from torch.utils.data import Subset, TensorDataset
from torch.utils.data.dataset import random_split
from torch import nn, optim
import torch.nn.functional as F
# from syft.frameworks.torch.differential_privacy import pate
# import syft as sy
import matplotlib.pyplot as plt

import os
import time
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.utils import resample, shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, precision_recall_curve, auc
from sklearn.metrics import (
    recall_score,
    f1_score,
    precision_score,
    accuracy_score,
    confusion_matrix,
    precision_recall_curve,
    average_precision_score,
     auc,
)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader
from imblearn.over_sampling import RandomOverSampler

#Filtering errors
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
%pip install imblearn

### Section 1: Download and load MNIST dataset

*   **Trainset**: divide among teachers without overlapping
*   **Testset**: split into private(labeled) and public(unlabeled) datasets; use models trained by teachers to label public dataset and train with the private dataset



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data  = pd.read_csv('/content/drive/MyDrive/BotNeTIoT-L01_label_NoDuplicates.csv', index_col = 0)
target_column = 'label'

num_classes = data[target_column].nunique()

# Add Upsampling
# Separate features and target
X = data.drop(columns=[target_column])
y = data[target_column]

# Perform upsampling (for synthetic minority over sampling() smote)
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X, y)
# put it in one area rebuild dataset
data = pd.concat([X_resampled, y_resampled], axis=1)

# # Label encode the target column(convert text to numbers)
# from sklearn.preprocessing import LabelEncoder

# # Instantiate LabelEncoder
# label_encoder = LabelEncoder()

# # Encode the target column
# data[target_column] = label_encoder.fit_transform(data[target_column])

# # Identify non-numeric columns (only two so dropping)
# print(data.select_dtypes(include='object').columns)

# data.drop(['proto', 'service'], axis=1, inplace=True)

Mounted at /content/drive


In [ ]:
# Separate features and target
y = data[target_column].values
X = data.drop(columns=[target_column]).values  #

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Define a custom dataset
class TabularDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


# Create datasets
trainset = TabularDataset(X_train, y_train)
testset = TabularDataset(X_test, y_test)

# Data loaders
train_loader = DataLoader(trainset, batch_size=32, shuffle=True)
test_loader = DataLoader(testset, batch_size=32, shuffle=False)

In [ ]:
# training and test data info
print('num training data: {0}'.format(len(trainset)))
print('num test data: {0}'.format(len(testset)))

num training data: 3252230
num test data: 573924


### Section 2: Functions


*   `teacher_data_allocator`: evenly allocate training dataset to the number of teachers as their private datasets
*   `private_public_allocator`: split test dataset as the student's private dataset and public dataset
*   `train`: model training with training and valid datasets
*   `get_label_testset`: get labels of the student's public dataset from all teachers
*   `add_global_noise`: add global noise to the labels that most teachers agree
*   `create_labeled_public_db`: extract images from the public dataset and combine these images with the new labels; create a new dataloader for this new dataset



In [ ]:
# divide unique training datasets among teachers
# split each dataset into training and valid sets
def teacher_data_allocator(trainset, batch_size=64, num_teachers=10, valid_train_split=0.3):
  data_per_teacher = len(trainset) // num_teachers    # round to the nearest integer
  train_loaders = []
  valid_loaders = []
  for i in range(num_teachers):
    if i == num_teachers - 1:
      ind = list(range(i * data_per_teacher, len(trainset)))
    else:
      ind = list(range(i * data_per_teacher, (i + 1) * data_per_teacher))
    data = Subset(trainset, ind)
    split = int(valid_train_split * len(data))
    valid_data, train_data = random_split(data, [split, len(data) - split])
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, shuffle=False)
    train_loaders.append(train_loader)
    valid_loaders.append(valid_loader)
  return train_loaders, valid_loaders

In [ ]:
# divide testset into private and public
def private_public_allocator(dataset, batch_size=64, split=0.7):
  split_ind = int(len(dataset) * split)

  # validset and testset
  # validset is smaller because dataset of each teacher is small
  private_ind = list(range(0, split_ind))
  public_ind = list(range(split_ind, len(dataset)))
  private_set = Subset(dataset, private_ind)
  public_set = Subset(dataset, public_ind)

  # validloader and testloader
  private_loader = torch.utils.data.DataLoader(private_set, batch_size=batch_size, shuffle=True)
  public_loader = torch.utils.data.DataLoader(public_set, batch_size=batch_size, shuffle=False)
  return private_loader, public_loader

In [ ]:
def train(model, criterion, optimizer, trainloader, validloader, epochs=30, patience = 5, plot=False):
    model = model
    criterion = criterion
    optimizer = optimizer
    epochs = epochs

    train_losses, valid_losses = [], []
    train_accuracies, valid_accuracies = [], []

    # Early stopping variables
    best_valid_loss = float('inf')
    epochs_no_improve = 0
    early_stop = False

    for e in range(epochs):
        if early_stop:
            print("Early stopping triggered")
            break

        running_loss = 0
        correct_train = 0
        total_train = 0

        # Training loop
        model.train()
        for inputs, labels in trainloader:
            optimizer.zero_grad()

            log_ps = model(inputs)
            loss = criterion(log_ps, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Track accuracy for training
            ps = torch.exp(log_ps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            correct_train += equals.sum().item()
            total_train += labels.size(0)

        # Validation loop
        valid_loss = 0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            model.eval()
            for inputs, labels in validloader:
                log_ps = model(inputs)
                valid_loss += criterion(log_ps, labels).item()

                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                correct_val += equals.sum().item()
                total_val += labels.size(0)

        # Calculate average losses and accuracies
        train_losses.append(running_loss / len(trainloader))
        valid_losses.append(valid_loss / len(validloader))
        train_accuracies.append(correct_train / total_train)
        valid_accuracies.append(correct_val / total_val)

        # Print progress
        print(f"Epoch: {e+1}/{epochs}.. "
              f"Training Loss: {running_loss / len(trainloader):.3f}.. "
              f"Validation Loss: {valid_loss / len(validloader):.3f}.. "
              f"Validation Accuracy: {correct_val / total_val:.3f}")

        # Early stopping check
        current_valid_loss = valid_loss / len(validloader)
        if current_valid_loss < best_valid_loss:
            best_valid_loss = current_valid_loss
            epochs_no_improve = 0  # Reset counter
            best_model_state = model.state_dict()  # Save best model
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f"Validation loss has not improved for {patience} epochs. Stopping early.")
            early_stop = True

    # Load the best model state before returning
    model.load_state_dict(best_model_state)

    if plot:
        return model, train_losses, valid_losses, train_accuracies, valid_accuracies
    else:
        return model


In [ ]:
# label testset by models
def get_label_testset(models, testloader):
  test_labels = []
  for model in models:
    test_label = []
    for inputs, _ in testloader:
      with torch.no_grad():
        log_ps = model(inputs)
        ps = torch.exp(log_ps)
      top_p, top_class = ps.topk(1, dim=1)
      test_label.append(top_class.squeeze().tolist())
    test_label = sum(test_label, [])
    test_labels.append(test_label)
  return test_labels

In [ ]:
# get predictions with global noise
def add_global_noise(preds, epsilon=0.1):
  labels_with_noise = []
  for pred in preds:    # go thru entries
    label_counts = np.bincount(pred, minlength=preds.shape[1])

    epsilon = epsilon
    beta = 1 / epsilon

    for i in range(len(label_counts)):
      label_counts[i] += np.random.laplace(0, beta, 1)

    new_label = np.argmax(label_counts) % num_classes
    labels_with_noise.append(new_label)
  return np.array(labels_with_noise)

In [ ]:
# create a new dataloader for the public dataset
# using the new labels added global noise
def create_labeled_public_db(dataloader, labels, batch_size=64):
  image_list = []
  for image, _ in dataloader:
    image_list.append(image)
  data = np.vstack(image_list)
  new_dataloader = list(zip(data, labels))
  new_dataloader = torch.utils.data.DataLoader(new_dataloader, shuffle=False, batch_size=batch_size)
  return new_dataloader

### Section 3: Define neural network model structure

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

input_size = X_train.shape[1]

# Teacher 1: LSTM
class TeacherLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=1, num_classes=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        if x.dim() == 2:
            x = x.unsqueeze(1)
        _, (hn, _) = self.lstm(x)
        out = self.fc(hn[-1])
        return F.log_softmax(out, dim=1)


# Teacher 2: MLP (your current one)
class TeacherMLP(nn.Module):
    def __init__(self, input_size, num_classes=2):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        out = self.fc3(x)
        return F.log_softmax(out, dim=1)


# Teacher 3: CNN
class TeacherCNN(nn.Module):
    def __init__(self, input_size, num_classes=2):
        super().__init__()
        # treat each feature vector as [channels=1, length=input_size]
        self.conv1 = nn.Conv1d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        # [batch, features] -> [batch, 1, features]
        if x.dim() == 2:
            x = x.unsqueeze(1)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.flatten(x, 1)          # flatten all except batch
        x = F.relu(self.fc1(x))
        out = self.fc2(x)
        return F.log_softmax(out, dim=1)



In [ ]:
class StudentGRU(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=1, num_classes=2):
        super().__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        if x.dim() == 2:
            x = x.unsqueeze(1)
        _, hn = self.gru(x)
        out = self.fc(hn[-1])
        return F.log_softmax(out, dim=1)

class StudentCNN(nn.Module):
    def __init__(self, input_size, num_classes=2, num_channels=64, kernel_size=3):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=num_channels, kernel_size=kernel_size, padding=1)
        self.bn1 = nn.BatchNorm1d(num_channels)
        self.conv2 = nn.Conv1d(in_channels=num_channels, out_channels=num_channels * 2, kernel_size=kernel_size, padding=1)
        self.bn2 = nn.BatchNorm1d(num_channels * 2)
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(num_channels * 2, num_classes)

    def forward(self, x):
        # If input is [batch, seq_len, features], permute for Conv1d: [batch, features, seq_len]
        if x.dim() == 3:
            x = x.permute(0, 2, 1)
        elif x.dim() == 2:
            x = x.unsqueeze(2)  # [batch, features, 1]

        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.global_pool(x).squeeze(-1)  # [batch, channels]
        out = self.fc(x)
        return F.log_softmax(out, dim=1)

class StudentMLP(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=1, num_classes=2):
        super().__init__()

        layers = []
        in_dim = input_size

        # Build hidden layers dynamically
        for _ in range(num_layers):
            layers.append(nn.Linear(in_dim, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.2))
            in_dim = hidden_size

        self.mlp = nn.Sequential(*layers)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # If input is [batch, seq_len, features], flatten seq_len dimension
        if x.dim() == 3:
            x = x.mean(dim=1)  # average pooling over time dimension
        out = self.mlp(x)
        out = self.fc(out)
        return F.log_softmax(out, dim=1)

### Section 4: Model training and PATE analysis
We first ask the teachers to train their data and use these models to label the student's public data. After adding the global noise to create the new labels for the public dataset, the student then trains his/her private (training set) and public (valid set) data. PATE analysis returns independent and dependent epsilons show how much information has been leaked and how much teachers agree with each other.

In [ ]:
# allocate training dataset to teachers
# split each of these datasets into training and valid datasets
teachers_train, teachers_valid = teacher_data_allocator(trainset)
print(len(teachers_train))
print(len(teachers_valid))

10
10


In [ ]:
# allocate test dataset to the student
# split the dataset into private and public datasets
private_loader, public_loader = private_public_allocator(testset)
print(len(private_loader))
print(len(public_loader))

6278
2691


In [ ]:
import torch.optim as optim

input_size = X_train.shape[1]

# Initialize teachers
teacher1 = TeacherMLP(input_size=input_size, num_classes=num_classes)
teacher2 = TeacherMLP(input_size=input_size, num_classes=num_classes)
teacher3 = TeacherMLP(input_size=input_size, num_classes=num_classes)

teacher4 = TeacherCNN(input_size=input_size, num_classes=num_classes)
teacher5 = TeacherCNN(input_size=input_size, num_classes=num_classes)
teacher6 = TeacherCNN(input_size=input_size, num_classes=num_classes)
teacher7 = TeacherCNN(input_size=input_size, num_classes=num_classes)

teacher8 = TeacherCNN(input_size=input_size, num_classes=num_classes)
teacher9 = TeacherCNN(input_size=input_size, num_classes=num_classes)
teacher10 = TeacherCNN(input_size=input_size, num_classes=num_classes)

teachers = [teacher1, teacher2, teacher3] # ,teacher4, teacher5, teacher6,teacher7, teacher8, teacher9, teacher10]

# Loss and optimizer for teachers
criterion = nn.NLLLoss()

teacher_models = []
for t in teachers:
    optimizer = optim.Adam(t.parameters(), lr=0.0001)
    teacher_model = train(t, criterion, optimizer, train_loader, test_loader, epochs=30)
    teacher_models.append(teacher_model)


# Teachers label the public dataset
teacher_preds = get_label_testset(teachers, public_loader)

# Aggregate with global noise
noisy_labels = add_global_noise(np.array(teacher_preds), epsilon=0.1)

Epoch: 1/30.. Training Loss: 0.004.. Validation Loss: 0.001.. Validation Accuracy: 1.000
Epoch: 2/30.. Training Loss: 0.001.. Validation Loss: 0.001.. Validation Accuracy: 1.000
Epoch: 3/30.. Training Loss: 0.001.. Validation Loss: 0.001.. Validation Accuracy: 1.000
Epoch: 4/30.. Training Loss: 0.001.. Validation Loss: 0.001.. Validation Accuracy: 1.000
Epoch: 5/30.. Training Loss: 0.001.. Validation Loss: 0.001.. Validation Accuracy: 1.000
Epoch: 6/30.. Training Loss: 0.001.. Validation Loss: 0.001.. Validation Accuracy: 1.000
Epoch: 7/30.. Training Loss: 0.001.. Validation Loss: 0.001.. Validation Accuracy: 1.000
Epoch: 8/30.. Training Loss: 0.001.. Validation Loss: 0.001.. Validation Accuracy: 1.000
Epoch: 9/30.. Training Loss: 0.001.. Validation Loss: 0.001.. Validation Accuracy: 1.000
Epoch: 10/30.. Training Loss: 0.001.. Validation Loss: 0.001.. Validation Accuracy: 1.000
Epoch: 11/30.. Training Loss: 0.001.. Validation Loss: 0.001.. Validation Accuracy: 1.000
Epoch: 12/30.. Trai

/tmp/ipython-input-269576571.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label_counts[i] += np.random.laplace(0, beta, 1)


In [ ]:
# predict labels for testset from each teacher
# row = num of test data
# col = predictions from each teacher
preds = get_label_testset(teacher_models, public_loader)
preds = np.array([np.array(p) for p in preds]).transpose(1, 0)

In [ ]:
preds_with_noise = add_global_noise(preds, epsilon=1.3)
len(preds_with_noise)

/tmp/ipython-input-269576571.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label_counts[i] += np.random.laplace(0, beta, 1)


172178

In [ ]:
# PATE Analysis manual
teacher_preds = preds
noise_eps = 1.3 # Privacy noise epsilon (scale parameter of Laplace noise)
delta = 1e-5 # Privacy delta (used in the privacy guarantee)



# Compute max votes for each query
max_votes = np.max(teacher_preds, axis=0) # Maximum votes for each query
total_queries = teacher_preds.shape[1]



# Data-independent epsilon
data_ind_eps = (1 / noise_eps) * np.max(max_votes)
print("Data-independent epsilon:", data_ind_eps)



# Data-dependent epsilon
total_votes = np.sum(teacher_preds, axis=0)
data_dep_eps = np.sum(
np.log(1 + (total_queries * max_votes) / (noise_eps * total_votes))
)
print("Data-dependent epsilon:", data_dep_eps)

Data-independent epsilon: 0.7692307692307692
Data-dependent epsilon: 8.033292351643122e-05


In [ ]:
# !pip install syft
# import syft as sy
# # PATE analysis
# data_dep_eps, data_ind_eps = sy.pate.perform_analysis(teacher_preds=preds.T, indices=preds_with_noise, noise_eps=0.1, delta=1e-5)
# print('Data dependent epsilon:', data_dep_eps)
# print('Data independent epsilon:', data_ind_eps)

In [ ]:
# create new dataloader for the public data
# using the original function
public_loader_labeled = create_labeled_public_db(public_loader, preds_with_noise)
len(public_loader_labeled)

2691

In [ ]:
# train model using private and public datasets
my_model = StudentMLP(input_size=input_size, num_classes=num_classes)
criterion = nn.NLLLoss()
optimizer = optim.Adam(my_model.parameters(), lr=0.0001)

my_model, train_losses, valid_losses, train_accuracies, valid_accuracies = train(
    model=my_model,
    criterion=criterion,
    optimizer=optimizer,
    trainloader=private_loader,
    validloader=public_loader_labeled,
    epochs=30, plot = True
)


## Student Plots

In [ ]:
# Plotting the losses
plt.figure(figsize=(12, 5))

# Plot Training and Validation Losses
plt.subplot(1, 2, 1)
plt.plot(range(1, len(train_losses)+1), train_losses, label='Training Loss', color='blue')
plt.plot(range(1, len(valid_losses)+1), valid_losses, label='Validation Loss', color='red')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(range(1, len(train_accuracies)+1), train_accuracies, label='Training Accuracy', color='blue')
plt.plot(range(1, len(valid_accuracies)+1), valid_accuracies, label='Validation Accuracy', color='red')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
print(valid_accuracies)

In [ ]:
def generate_predictions(model, validloader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in validloader:
            log_ps = model(inputs)
            ps = torch.exp(log_ps)
            top_p, top_class = ps.topk(1, dim=1)
            all_preds.extend(top_class.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return np.array(all_preds), np.array(all_labels)

# Get predictions and true labels
all_preds, all_labels = generate_predictions(my_model, public_loader_labeled)

# Calculate F1 score
f1 = f1_score(all_labels, all_preds, average='macro')

# Print F1 score
print(f"F1 Score: {f1:.4f}")

# Generate confusion matrix
conf_matrix = confusion_matrix(all_labels, all_preds)

# Plot confusion matrix with class names
class_labels = data[target_column].unique().tolist() #label_encoder.classes_
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()